## This script loads pickles of EPT files, calculates YF and median age YF, then makes 1:1 plots to compare simplified models to the most complex model.  The goal is to evaluate if and how complexity influences age metrics across scales.

### This version is slightly altered to limit the analyses

#### The script reads in the model_grid.csv file for each model and uses that to tie each particle's initial location to select categories, such as the HUC, NLCD, coarse fraction, etc.  The model_grid.csv file was created via verion 2 of the general models / GRTD notebooks, tied to PFJ's repo called "GenMod" on the USGS GitLab site.

In [1]:
__author__ = 'Paul Juckem'
%matplotlib notebook
import os, sys
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
import matplotlib.cm as cm
from datetime import datetime
import gdal
from gdal import ogr, osr
import gen_mod_functions as gm
import flopy as fp
import pickle
from ipywidgets import interact, Dropdown, Text
from IPython.display import display
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import csv

import fit_parametric_distributions

try:
    import rasterio
except:
    print('Install rasterio into your python environment. Or, skip plotting of geotiffs at the end of this notebook')

modifier = False
def ReturnEntry(sender):
    modifier.value = intext.value

## Specify user input, including list of models to compare and which axes to plot them, plus attributes to analyze.

In [ ]:
#simulate_list = ['FWP1L_zK', 'FWP5L_zK', 'FWP5L_hK']  # list of models (w/ matching directory name) to analyze
simulate_list = ['FWP5L_hK']

# This variable facilitates scenario testing (stopping or not at weak sinks/sources; porosity values & configuration, etc.)
#scenario_name = None  # set to None if no additional text was added to the modpath file names.
#scenario_name = 'passthroughsnk'  # an optional text string added to MP file names. MUST match value used in NB 01aa!!!
scenario_name = 'het_n'  

# when comparing results of the scenario to the base case, which weighting option to use? 'flux' or 'volume'
base_case = 'flux'

# for plotting 1:1 graphs
x1, y1 = 0, 1  # index of names in the yet-to-be-populated eptlist variable.  (0=base case; 1 = scenario)

In [ ]:
surf_aq_lays = 3  # deepest layer of the surficial aquifer.

nrow, ncol = 930, 650  # easier to hardcode this than waste the time to read-in a model just to get DIS info.

# for labeling RTD plots
label_by_model_name = True  # if false, will include zone info in label.

category = 'HUC12' 

minptl = 500  # The minimum number of particles for EACH model within each HUC (eg: if FWP5L has 2000, 
               # but FWP1L has 800, none get plotted in the 1:1 plots.  Still included in the RTD plots.)
               # 500-1000 seems reasonable for 1:1 plots as it includes only HUCs with really refined RTDs;
               # however, 100 - 500 seems more reasonable if we want to visualize spatial patterns because it
               # allows more HUCs to be plotted.  100-500 is based on visually inspecting RTD curves.
                
# Columns in the model_grid.csv file to keep.  Purge all others.
mg_columns = ['node_num', 'row', 'col', 'HUC6', 'HUC8', 'HUC10', 'HUC12', 'ibound', 'gage_id', 'coarse_flag', 
              'qu_atlas', 'catchment', 'ssurgo', 'stream_order', 'surfmat']
                
purge_hucs = [40602, 4060200, 406020000, 40602000000]  # all hucs for Lake Michigan      

HUCshpdict = {'HUC6':'E:/HUCS/WBD_4n7/WBD_HUC6_UTMft_FWPdomain.shp', 
              'HUC8':'E:/HUCS/WBD_4n7/WBD_HUC8_UTMft_FWPdomain.shp', 
              'HUC10':'E:/HUCS/WBD_4n7/WBD_HUC10_UTMft_FWPdomain.shp', 
              'HUC12':'E:/HUCS/WBD_4n7/WBD_HUC12_UTMft_FWPdomain.shp'}

HUCtiffdict = {'HUC6':'E:/HUCS/WBD_4n7/HUC6_UTMft_FWP.tiff', 
              'HUC8':'E:/HUCS/WBD_4n7/HUC8_UTMft_FWP.tiff', 
              'HUC10':'E:/HUCS/WBD_4n7/HUC10_UTMft_FWP.tiff',
              'HUC12':'E:/HUCS/WBD_4n7/HUC12_UTMft_FWP.tiff'}

#HUCproprast = './vK_lay1_hK-vK.tif'
HUCproprastlist = ['../TIFFs/vK_lay1_5h-5z.tif', '../TIFFs/vK_lay2_5h-5z.tif', '../TIFFs/vK_lay3_5h-5z.tif', '../TIFFs/vK_lay1_hK-vK.tif',
                 '../TIFFs/vani_lay1_5h-5z.tif', '../TIFFs/vani_lay2_5h-5z.tif', '../TIFFs/vani_lay3_5h-5z.tif',
                 '../TIFFs/T_lay1_5h-5z.tif', '../TIFFs/T_lay2_5h-5z.tif', '../TIFFs/T_lay3_5h-5z.tif', '../TIFFs/GlacT_5h-5z.tif',
                 '../TIFFs/glac_satthick_5h-5z.tif', '../TIFFs/BrRCH_5h-5z.tif', '../TIFFs/RCH_5h-5z.tif', 
                 '../TIFFs/hK_lay1_5h-5z.tif', '../TIFFs/hK_lay2_5h-5z.tif', '../TIFFs/hK_lay3_5h-5z.tif',
                 '../TIFFs/BrT_5h-5z.tif', '../TIFFs/glacT2BrT_5h-5z.tif', '../TIFFs/RCHUZF_5h-5z.tif', '../TIFFs/UZF_5h-5z.tif', 
                 '../TIFFs/RCHcbb_5h-5z.tif', '../TIFFs/rch-over-satKs_5h-5z.tif', '../TIFFs/sat-weighted_Ks_5h-5z.tif']
HUCfluxrastlist = ['../TIFFs/SWleak_5h-5z.tif', '../TIFFs/MNW2_5h-5z.tif', '../TIFFs/SFR_5h-5z.tif']
genHUCdict = {'Oconto':'04030104', 'TWR':'0403020218'}

#### Prep the script for the models to be analyzed

In [ ]:
homes = ['../Models']
fig_dir = '../Figures'

if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)

mfpth = '../executables/MODFLOW-NWT_1.0.9/bin/MODFLOW-NWT_64.exe'
mp_exe_name = '../executables/modpath.6_0/bin/mp6x64.exe' 

age_cutoff = 65
#year_cutoff = '01/01/1952'

surf_aq_lays = 3  # deepest layer of the surficial aquifer.

dir_list = []
modlist = []
i = 0
r = 0

path_dict = {}
dfdict = {}
totp = {}
for home in homes:
    if os.path.exists(home):
        for dirpath, dirnames, filenames in os.walk(home):
            for f in filenames:
                if os.path.splitext(f)[-1] == '.nam':
                    mod = os.path.splitext(f)[0]
                    i += 1
                    if mod in simulate_list:
                        modlist.append(mod)
                        dir_list.append(dirpath)
                        r += 1
                        path_dict[mod] = dirpath
                               
print('    {} models read'.format(i))
print('These {} models will be analyzed: {}'.format(r, modlist))

In [ ]:
# generate list of nam files:
nam_list = []
for pth in dir_list:
    model = os.path.normpath(pth).split(os.sep)[2]
    nam_file = '{}.nam'.format(model)
    nam_list.append(nam_file)

In [ ]:
# setup dictionaries of the MODFLOW units for proper labeling of figures.
lenunit = {0:'undefined units', 1:'feet', 2:'meters', 3:'centimeters'}
timeunit = {0:'undefined', 1:'second', 2:'minute', 3:'hour', 4:'day', 5:'year'}

# Create dictionary of multipliers for converting model time units to days
time_dict = dict()
time_dict[0] = 1.0 # undefined assumes days, so enter conversion to days
time_dict[1] = 24 * 60 * 60
time_dict[2] = 24 * 60
time_dict[3] = 24
time_dict[4] = 1.0
time_dict[5] = 1.0

In [ ]:
# Read-in model info and check max/min nlay & create list of DIS objects.  
# Assumes all else is the same among models (hnoflow, hdry, etc)
print ('Reading model information')
nlay_min = 100
nlay_max = 0

dis_objs = []
for i, model in enumerate(nam_list):
    nam_file = model
    model_ws = dir_list[i]
    
    fpmg = fp.modflow.Modflow.load(nam_file, model_ws=model_ws, exe_name=mfpth, version='mfnwt', 
                                   load_only=['DIS', 'BAS6', 'UPW', 'OC'], check=False)

    dis = fpmg.get_package('DIS')
    dis_objs.append(dis)
    bas = fpmg.get_package('BAS6')
    upw = fpmg.get_package('UPW')
    oc = fpmg.get_package('OC')

    delr = dis.delr
    delc = dis.delc
    nlay = dis.nlay
    nrow = dis.nrow
    ncol = dis.ncol
    bot = dis.getbotm()
    top = dis.gettop()

    hnoflo = bas.hnoflo
    ibound = np.asarray(bas.ibound.get_value())
    hdry = upw.hdry
    
    if nlay > nlay_max:
        nlay_max = nlay
    if nlay < nlay_min:
        nlay_min = nlay
        
    print('  .. done reading model {}'.format(i+1))

print ('   ... all done') 

print('minimum layers in a model:  {}'.format(nlay_min))
print('maximum layers in a model:  {}'.format(nlay_max))

## Process endpoint files and merge with model_grid.csv

In [ ]:
def purge(ep_data):
    pre_Quaternary = ep_data.loc[ep_data.rt>=2.6e6]
    pre_Cretaceous = ep_data.loc[ep_data.rt>=66e6]
    preCambrian = ep_data.loc[ep_data.rt>=541e6]
    pre_earth = ep_data.loc[ep_data.rt>=4.6e9]

    print('\nFor your information:')
    print('{} particles were simulated as being older than Earth!'.format(preCambrian.shape[0]))
    print('{} particles were simulated as being PreCambrian in age.'.format(preCambrian.shape[0]))
    print('{} particles were simulated as being Cretaceous in age or older.'.format(pre_Cretaceous.shape[0]))
    print('{} particles were simulated as being pre-Quaternary in age.'.format(pre_Quaternary.shape[0]))
    
    ep_data = ep_data.loc[ep_data.rt<4.6e9]
    print('Purged particles older than earth')
    return(ep_data)

In [ ]:
# read-in the model_grid.csv file for each model.  Then create a dataframe from each csv & pickle file.

dfdict = {}
totp = {}

for i, model in enumerate(modlist):
    g = os.path.join(path_dict[model], 'model_grid.csv')
    z = os.path.join(path_dict[model], 'zone_df.csv')
    try:
        df = pd.read_csv(g)
        df.ibound.replace(0, np.nan, inplace=True)
        df = df[df.ibound.notnull()]
        df = df[mg_columns]  # keep just the desired fields
        # re-calculate 2D cell number b/c node_num is computed differently for model_grid.csv and what's in the ept file.
        df['cellnum2d'] = df.row * ncol + df.col
        
        zone_df = pd.read_csv(z, index_col=0)
        for group in zone_df:
            print('\nReading-in the EPT file for {} in {}'.format(group, model))
            if scenario_name == None:
                print("Oops, I didn't really plan for this.  Stopping")
                sys.exit()
            else:
                p1 = os.path.join(path_dict[model], '{}_{}_{}.mpend'.format(model, base_case, group))
                p2 = os.path.join(path_dict[model], '{}_{}_{}_{}.mpend'.format(model, base_case, group, scenario_name))
                mnz1 = '{}_{}'.format(model, group)
                mnz2 = '{}_{}_{}'.format(model, group, scenario_name)
                eptu1 = fit_parametric_distributions.read_endpoints(p1, dis, time_dict)
                eptu2 = fit_parametric_distributions.read_endpoints(p2, dis, time_dict)
                
            eptu1 = purge(eptu1)
            eptu2 = purge(eptu2)
            eptu1['cellnum2d'] = (eptu1['Initial Row']-1) * ncol + (eptu1['Initial Column'] -1)  # -1 to convert to 0-based
            eptu2['cellnum2d'] = (eptu2['Initial Row']-1) * ncol + (eptu2['Initial Column'] -1)  # -1 to convert to 0-based
            eptu1_mg = eptu1.join(df, on='cellnum2d', lsuffix='_ept', rsuffix='_mg')
            eptu2_mg = eptu2.join(df, on='cellnum2d', lsuffix='_ept', rsuffix='_mg')
            eptu1_mg = eptu1_mg[eptu1_mg['Initial Layer'] <= surf_aq_lays]  # ensure that we're only analyzing Glacial!
            eptu2_mg = eptu2_mg[eptu2_mg['Initial Layer'] <= surf_aq_lays]  # ensure that we're only analyzing Glacial!

            #dfdict[model] = eptu_mg
            dfdict[mnz1] = eptu1_mg
            dfdict[mnz2] = eptu2_mg
            totp[mnz1] = eptu1_mg.rt.count()
            totp[mnz2] = eptu2_mg.rt.count()
            
            x1 = eptlist[x1]  # pull out the file name after supplying the index
            y1 = eptlist[y1]
            
    except (AttributeError, ValueError, IOError, IndexError):
        print('ERROR. THIS CODE BLOCK DID NOT COMPLETE. TROUBLE-SHOOT AND TRY AGAIN')
        print('The error occured while working on this model: {}'.format(model))
        raise SystemExit()

eptlist = list(dfdict.keys())
print('....done')

In [ ]:
print(eptlist)
print(modlist)

In [ ]:
np.all(dfdict[eptlist[0]] == dfdict[eptlist[1]])

In [ ]:
# Plot age distributions for all 3 models by HUC ID, for selected HUC scale.

#uniques = dfdict[modlist[0]][category].unique()
uniques = dfdict[eptlist[0]][category].unique()
# remove any HUCs listed in purge_hucs
for h in purge_hucs:
    ind = np.where(uniques==h)
    uniques = np.delete(uniques, ind)

n_uni = len(uniques)
sum_p = {}
for mn in eptlist:
    sum_p[mn] = 0
    
if n_uni <= 20:
    vplots = int(np.ceil(n_uni/ 3.0))
    figsize = (12, 3*vplots)
    CS, CSaxes = plt.subplots(vplots, 3, figsize=figsize)
else:
    hplots = int(np.round(np.sqrt(n_uni)))
    vplots = int(np.ceil(np.sqrt(n_uni)))
    figsize = (hplots*4, hplots*3)
    CS, CSaxes = plt.subplots(vplots, hplots, figsize=figsize)
        
colors_line = plt.cm.brg(np.linspace(0, 1, len(eptlist)))

for ax, cat_val in zip(CSaxes.flat, uniques):
    n = []
    for i, md in enumerate(eptlist):
        rt = dfdict[md].loc[dfdict[md][category]==cat_val, 'rt']  # 'rt' is "raw time" in the dataframe
        rt.sort_values(inplace=True)
        n.append(rt.count())
        sum_p[md] = sum_p[md] + rt.count()
        y_rt = np.linspace(0, 1, rt.shape[0])
    
        if label_by_model_name:
            label = eptlist[i]
        else:
            label = md
        ax.plot(rt, y_rt, c=colors_line[i], label=label)
        ax.plot((age_cutoff, age_cutoff), (0.2, 1), 'k--')
        
        title = '{}: {}'.format(category, cat_val)
        ax.set_title(title, fontsize=12)

        ax.set_xscale('log')
        ax.set_xlim(1e0, 1e3)
        ax.set_ylim(0, )

        ax.legend(loc=0, frameon=False, fontsize=8)#, bbox_to_anchor=(0.20, 0.2), ncol=1)
        ax.set_xlabel('Residence time, in years')
        ax.set_ylabel('Cumulative frequency')
        if len(n) == len(modlist):
            nmin, nmax = min(n), max(n)
            ax.text(5,0.02, '# particles: {:,} - {:,}'.format(nmin, nmax))
        
CS.suptitle('Comparison of glacial particle time distributions by {} for FWP scenarios'.format(category), fontsize=18)  
CS.tight_layout()

if n_uni < 18:
    #CS.subplots_adjust(top= 0.86, hspace=0.85)
    CS.subplots_adjust(top= 0.86)
elif n_uni < 100:
    CS.subplots_adjust(top= 0.93, hspace=0.55)
elif n_uni < 400:
    CS.subplots_adjust(top= 0.95, hspace=0.55)
else:
    CS.subplots_adjust(top= 0.97, hspace=0.55)

dst = 'RTD_compare--{}_{}'.format(category, scenario_name)
dst_pth = os.path.join(fig_dir, dst)
plt.savefig(dst_pth)
#plt.close()

In [ ]:
# 1:1 plot of Young Fraction btwn 5LhK against 1L and 5LzK models for select HUC scale.

uniques = dfdict[eptlist[0]][category].unique()
# remove any HUCs listed in purge_hucs
for h in purge_hucs:
    ind = np.where(uniques==h)
    uniques = np.delete(uniques, ind)

yfhucdict = {}
yfvalues = []
skiphuc = []
for j, cat_val in enumerate(uniques):  # each HUC ID
    yfmoddict = {}
    for i, md in enumerate(eptlist):  # each of the 3 FWP models
        #mn = modlist[i]
        mn = eptlist[i]
        df = dfdict[md][dfdict[md][category]==cat_val].copy()
        if df.rt.count() >= minptl:
            youngdf = df.loc[df.rt < age_cutoff]
            yf = youngdf.rt.count() / df.rt.count()
            yfmoddict[mn] = yf
            yfvalues.append(yf)
        else:
            skiphuc.append(cat_val)
            break
    if cat_val not in skiphuc:
        yfhucdict[cat_val] = yfmoddict    
        
ddd = pd.DataFrame(yfhucdict).T
print(mean_squared_error(np.array(ddd[x1]), np.array(ddd[y1])))
r2_y1 = r2_score(np.array(ddd[x1]), np.array(ddd[y1]))
#r2_y2 = r2_score(np.array(ddd[x1]), np.array(ddd[y2]))
print(r2_y1)
#print(r2_y2)

ax = ddd.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', label='1-layer zoned K')
#ddd.plot(kind='scatter', x=x1, y=y2, marker='^', color='blue', label = '5-layer zoned K', ax=ax)
mini, maxi = min(yfvalues), max(yfvalues)
y1x = maxi - ((maxi-mini)/2)
ax.plot((mini, maxi), (mini, maxi), 'k--')
plt.xlabel('Fraction of young water (<{} yr) in the base-case model'.format(age_cutoff))
plt.ylabel('Fraction of young water (<{} yr) in the scenario models'.format(age_cutoff))
plt.suptitle('Comparison of glacial young fraction\n among model scenarios by {}'.format(category), fontsize=14)
ax.text(0.76, maxi, 'R2 = {:3.3f}'.format(r2_y1), fontsize=9, color='green')
#ax.text(0.76, maxi-0.02, 'R2 = {:3.3f}'.format(r2_y2), fontsize=9, color='blue')
#plt.tight_layout()

dst = 'YFrac_glac_121_{}_{}'.format(category, scenario_name)
#loc = os.path.dirname(path[modlist[0]])  # should go up one directory to the dir that houses all of the models.
#dst_pth = os.path.join(loc, dst)
dst_pth = os.path.join(fig_dir, dst)
plt.savefig(dst_pth)
#plt.close()

In [ ]:
# 1:1 plot of Young Fraction btwn 5LhK against 1L and 5LzK models for ALL HUC scales.

hucs = ['HUC6', 'HUC8', 'HUC10', 'HUC12']
nhucs = len(hucs)
fig, [[ax1, ax2], [ax3, ax4]] = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
axisdict = {0:ax1, 1:ax2, 2:ax3, 3:ax4}

for k, each in enumerate(hucs):
    # make sure we get ALL unique HUC IDs
    u1 = dfdict[eptlist[0]][each].unique()
    u2 = dfdict[eptlist[1]][each].unique()
    #u3 = dfdict[eptlist[2]][each].unique()
    u = np.append(u1, u2)
    #u = np.append(u, u3)
    uniques = np.unique(u)
    # remove any HUCs listed in purge_hucs
    for h in purge_hucs:
        ind = np.where(uniques==h)
        uniques = np.delete(uniques, ind)
    axis = axisdict[k]

    yfhucdict = {}
    yfvalues = []
    skiphuc = []
    for j, cat_val in enumerate(uniques):  # each HUC ID
        yfmoddict = {}
        for i, md in enumerate(eptlist):  # each of the 3 FWP models
            mn = eptlist[i]
            df = dfdict[md].loc[dfdict[md][each]==cat_val]
            if df.rt.count() >= minptl:
                youngdf = df.loc[df.rt < age_cutoff]
                yf = youngdf.rt.count() / df.rt.count()
                yfmoddict[mn] = yf
                yfvalues.append(yf)
            else:
                skiphuc.append(cat_val)
                break
        if cat_val not in skiphuc:
            yfhucdict[cat_val] = yfmoddict    

    ddd = pd.DataFrame(yfhucdict).T
    r2_y1 = r2_score(np.array(ddd[x1]), np.array(ddd[y1]))
    #r2_y2 = r2_score(np.array(ddd[x1]), np.array(ddd[y2]))
    
    #plotting
    mini, maxi = min(yfvalues), max(yfvalues)
    #xplot = mini+(maxi-mini)/2
    yrange = maxi-mini
    if k == 0:
        ddd.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', label='1-layer zoned K', ax=axis)
        #ddd.plot(kind='scatter', x=x1, y=y2, marker='^', color='blue', label = '5-layer zoned K', ax=axis)
        axis.text(mini, mini+0.83*yrange, 'R2 = {:3.3f}'.format(r2_y1), fontsize=9, color='green')
        #axis.text(mini, mini+0.78*yrange, 'R2 = {:3.3f}'.format(r2_y2), fontsize=9, color='blue')
    else:
        ddd.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', ax=axis)
        #ddd.plot(kind='scatter', x=x1, y=y2, marker='^', color='blue', ax=axis)
        axis.text(mini, mini+yrange, 'R2 = {:3.3f}'.format(r2_y1), fontsize=9, color='green')
        #axis.text(mini, mini+0.95*yrange, 'R2 = {:3.3f}'.format(r2_y2), fontsize=9, color='blue')
    
    axis.plot((mini, maxi), (mini, maxi), 'k--')
    axis.set_xlabel(''), axis.set_ylabel('')
    axis.set_title(each)
    
    # Add an attribute to the HUC shapefile
    shapefile = HUCshpdict[each]
    shp = gpd.read_file(shapefile)  
    shp[each] = shp[each].astype('int64')  # convert so can merge
    ddd.index = ddd.index.astype('int64')  # convert so can merge
    ddd['YF_{}-{}'.format(scenario_name, base_case)] = ddd[x1] - ddd[y1]
    #ddd['YF_hK-zK'] = ddd[x1] - ddd[y2]

    df = pd.merge(shp, ddd, how='outer', left_on=each, right_index=True)
    dst = shapefile[:-4] + '_YF_{}.shp'.format(scenario_name)
    df.to_file(dst)

fig.text(0.5, 0.06, 'Fraction of young water (<{} yr) in the base model'.format(age_cutoff), ha='center')
fig.text(0.06, 0.48, 'Fraction of young water (<{} yr) in the scenario model'.format(age_cutoff), va='center', rotation='vertical')
plt.subplots_adjust(top = .9)
fig.suptitle('Comparison of glacial aquifer\n young fractions by HUCs for scenario "{}"'.format(scenario_name), fontsize=14)

dst = 'YFrac_glac_121_allHUCs_{}'.format(scenario_name)
#loc = os.path.dirname(path[modlist[0]])  # go up one directory to the dir that houses all of the models.
#dst_pth = os.path.join(loc, dst)
dst_pth = os.path.join(fig_dir, dst)
plt.savefig(dst_pth)
#plt.close()

In [ ]:
# 1:1 plot of meanYoungAge and meanOldAge btwn 5LhK against 1L and 5LzK models for select HUC scale.

# make sure we get ALL unique HUC IDs
u1 = dfdict[eptlist[0]][category].unique()
u2 = dfdict[eptlist[1]][category].unique()
#u3 = dfdict[eptlist[2]][category].unique()
u = np.append(u1, u2)
#u = np.append(u, u3)
uniques = np.unique(u)
# remove any HUCs listed in purge_hucs
for h in purge_hucs:
    ind = np.where(uniques==h)
    uniques = np.delete(uniques, ind)

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 4.6))

skiphuc = []
yfhucdict = {}
yfvalues = []
ofhucdict = {}
ofvalues = []
for j, cat_val in enumerate(uniques):  # each HUC ID
    yfmoddict = {}
    ofmoddict = {}
    for i, md in enumerate(eptlist):  # each of the 3 FWP models
        mn = eptlist[i]
        df = dfdict[md].loc[dfdict[md][category]==cat_val]
        if df.rt.count() >= minptl:
            youngdf = df.loc[df.rt < age_cutoff]
            olddf = df.loc[df.rt >= age_cutoff]
            yfage = youngdf.rt.median()
            ofage = olddf.rt.median()
            yfmoddict[mn] = yfage
            yfvalues.append(yfage) 
            ofmoddict[mn] = ofage
            ofvalues.append(ofage)
        else:
            skiphuc.append(cat_val)
            break
    if cat_val not in skiphuc:
        yfhucdict[cat_val] = yfmoddict
        ofhucdict[cat_val] = ofmoddict
        
young = pd.DataFrame(yfhucdict).T
old = pd.DataFrame(ofhucdict).T
young.dropna(axis='index', how='any', inplace=True)
old.dropna(axis='index', how='any', inplace=True)
r2_y1y = r2_score(np.array(young[x1]), np.array(young[y1]))
#r2_y2y = r2_score(np.array(young[x1]), np.array(young[y2]))
r2_y1o = r2_score(np.array(old[x1]), np.array(old[y1]))
#r2_y2o = r2_score(np.array(old[x1]), np.array(old[y2]))

#plotting
miny, maxy = min(yfvalues), max(yfvalues)
mino, maxo = min(ofvalues), max(ofvalues)
xplot = mini+(maxi-mini)/2
yrange_y = maxy-miny
yrange_o = maxo-mino

young.plot(kind='scatter', x=x1, y=y1, marker='o', c='green', label='1-layer zoned K', ax=ax1)
#young.plot(kind='scatter', x=x1, y=y2, marker='^', c='blue', label='5-layer zoned K', ax=ax1)
old.plot(kind='scatter', x=x1, y=y1, marker='o', c='green', ax=ax2)
#old.plot(kind='scatter', x=x1, y=y2, marker='^', c='blue', ax=ax2)
ax1.text(miny, miny+0.80*yrange_y, 'R2 = {:3.3f}'.format(r2_y1y), fontsize=9, color='green')
#ax1.text(miny, miny+0.75*yrange_y, 'R2 = {:3.3f}'.format(r2_y2y), fontsize=9, color='blue')
ax2.text(mino, mino+yrange_o, 'R2 = {:3.3f}'.format(r2_y1o), fontsize=9, color='green')
#ax2.text(mino, mino+0.95*yrange_o, 'R2 = {:3.3f}'.format(r2_y2o), fontsize=9, color='blue')
ax1.plot((miny, maxy), (miny, maxy), 'k--')
ax2.plot((mino, maxo), (mino, maxo), 'k--')
ax1.set_xlabel(''), ax1.set_ylabel(''), ax2.set_ylabel(''), ax2.set_xlabel('')
fig.text(0.25, 0.82, 'Young fraction')
fig.text(0.73, 0.82, 'Old fraction', ha='center')
fig.text(0.07, 0.45, 'Median age (yrs) for the scenario model', va='center', rotation='vertical')
fig.text(0.5, 0.01, 'Median age (yrs) for the base model', ha='center')
plt.subplots_adjust(top = .8)
fig.suptitle('Comparison of median ages for young and\n old water in the glacial aquifer by {} for scenario {}'.format(category, scenario_name), fontsize=14)
#fig.tight_layout()

dst = 'YnOages_121_{}_{}'.format(category, scenario_name)
#loc = os.path.dirname(path[modlist[0]])  # should go up one directory to the dir that houses all of the models.
#dst_pth = os.path.join(loc, dst)
dst_pth = os.path.join(fig_dir, dst)
plt.savefig(dst_pth)
#plt.close()

In [ ]:
# 1:1 plot of meanYoungAge and meanOldAge btwn 5LhK against 1L and 5LzK models for ALL HUC scales.

# Better viewed as separate plots via code block below
'''
hucs = ['HUC6', 'HUC8', 'HUC10', 'HUC12']
nhucs = len(hucs)
fig, [[ax1, ax2, ax3, ax4], [ax5, ax6, ax7, ax8]] = plt.subplots(nrows=2, ncols=4, figsize=(20, 10))
youngaxisdict = {0:ax1, 1:ax2, 2:ax5, 3:ax6}
oldaxisdict = {0:ax3, 1:ax4, 2:ax7, 3:ax8}

for k, each in enumerate(hucs):
    # make sure we get ALL unique HUC IDs
    u1 = dfdict[eptlist[0]][each].unique()
    u2 = dfdict[eptlist[1]][each].unique()
    u3 = dfdict[eptlist[2]][each].unique()
    u = np.append(u1, u2)
    u = np.append(u, u3)
    uniques = np.unique(u)
    # remove any HUCs listed in purge_hucs
    for h in purge_hucs:
        ind = np.where(uniques==h)
        uniques = np.delete(uniques, ind)
    youngaxis = youngaxisdict[k]
    oldaxis = oldaxisdict[k]
    ofhucdict = {}
    yfhucdict = {}
    yfvalues = []
    ofvalues = []
    skiphuc = []
    for j, cat_val in enumerate(uniques):  # each HUC ID
        yfmoddict = {} 
        ofmoddict = {} 
        for i, md in enumerate(eptlist):  # each of the 3 FWP models
            mn = modlist[i]
            df = dfdict[md].loc[dfdict[md][each]==cat_val]
            if df.rt.count() >= minptl:
                youngdf = df.loc[df.rt < age_cutoff]
                olddf = df.loc[df.rt >= age_cutoff]
                yfage = youngdf.rt.median()
                ofage = olddf.rt.median()
                yfmoddict[mn] = yfage
                yfvalues.append(yfage) 
                ofmoddict[mn] = ofage
                ofvalues.append(ofage)
                # pull out raw time info for generalized model areas for later use...
                if (cat_val == float(genHUCdict['Oconto'])) and ('fwp5lzk' in mn):  # 5lzK is the most similar to Generalized models
                    FWPzKocontoDF = df.copy()
                #elif cat_val == float(genHUCdict['TWR'])  and (md == 'fwp5lzk'):
                elif cat_val == float(genHUCdict['TWR'])  and ('fwp5lzk' in mn):
                    FWPzKtwrDF = df.copy()
                elif (cat_val == float(genHUCdict['Oconto'])) and ('fwp5lhk' in mn):  # 5lzK is the most similar to Generalized models
                    FWPhKocontoDF = df.copy()
                elif cat_val == float(genHUCdict['TWR'])  and ('fwp5lhk' in mn):
                    FWPhKtwrDF = df.copy()
                elif (cat_val == float(genHUCdict['Oconto'])) and ('fwp1l' in mn):  # 5lzK is the most similar to Generalized models
                    FWP1locontoDF = df.copy()
                elif cat_val == float(genHUCdict['TWR'])  and ('fwp1l' in mn):
                    FWP1ltwrDF = df.copy()                    
            else:
                skiphuc.append(cat_val)
                break
        if cat_val not in skiphuc:
            yfhucdict[cat_val] = yfmoddict
            ofhucdict[cat_val] = ofmoddict            
            
    young = pd.DataFrame(yfhucdict).T
    old = pd.DataFrame(ofhucdict).T
    for n in modlist:
        young = young.loc[~young[n].isnull()]  # need to remove any NANs
        old = old.loc[~old[n].isnull()]  # need to remove any NANs
    r2_y1y = r2_score(np.array(young[x1]), np.array(young[y1]))
    r2_y2y = r2_score(np.array(young[x1]), np.array(young[y2]))
    r2_y1o = r2_score(np.array(old[x1]), np.array(old[y1]))
    r2_y2o = r2_score(np.array(old[x1]), np.array(old[y2]))
    
    #plotting
    miny, maxy = min(yfvalues), max(yfvalues)
    mino, maxo = min(ofvalues), max(ofvalues)
    #xplot = mini+(maxi-mini)/2
    yrange_y = maxy-miny
    yrange_o = maxo-mino
    
    if k == 0:
        young.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', label='1-layer zoned K', ax=youngaxis)
        young.plot(kind='scatter', x=x1, y=y2, marker='^', color='blue', label = '5-layer zoned K', ax=youngaxis)        
        old.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', ax=oldaxis)
        old.plot(kind='scatter', x=x1, y=y2, marker='^', color='blue', ax=oldaxis)  
    else:
        young.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', ax=youngaxis)
        young.plot(kind='scatter', x=x1, y=y2, marker='^', color='blue', ax=youngaxis)
        old.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', ax=oldaxis)
        old.plot(kind='scatter', x=x1, y=y2, marker='^', color='blue', ax=oldaxis)  
        
    youngaxis.text(miny, miny+yrange_y, 'R2 = {:3.3f}'.format(r2_y1y), fontsize=9, color='green')
    youngaxis.text(miny, miny+0.95*yrange_y, 'R2 = {:3.3f}'.format(r2_y2y), fontsize=9, color='blue')
    oldaxis.text(mino, mino+yrange_o, 'R2 = {:3.3f}'.format(r2_y1o), fontsize=9, color='green')
    oldaxis.text(mino, mino+0.95*yrange_o, 'R2 = {:3.3f}'.format(r2_y2o), fontsize=9, color='blue')
    youngaxis.plot((miny, maxy), (miny, maxy), 'k--')
    oldaxis.plot((mino, maxo), (mino, maxo), 'k--')

    youngaxis.set_xlabel(''), youngaxis.set_ylabel(''), oldaxis.set_xlabel(''), oldaxis.set_ylabel('')
    youngaxis.set_title(each)
    oldaxis.set_title(each)
    
    # Add an attribute to the HUC shapefile
    shapefile = HUCshpdict[each]
    shp = gpd.read_file(shapefile)  
    shp[each] = shp[each].astype('int64')  # convert so can merge
    
    young.index = young.index.astype('int64')  # convert so can merge
    young['mYage_hK-1L'] = young[x1] - young[y1]
    young['mYage_hK-zK'] = young[x1] - young[y2]
    df = pd.merge(shp, young, how='outer', left_on=each, right_index=True)
    dst = shapefile[:-4] + '_mYage.shp'
    df.to_file(dst)
    
    old.index = old.index.astype('int64')  # convert so can merge
    old['mOage_hK-1L'] = old[x1] - old[y1]
    old['mOage_hK-zK'] = old[x1] - old[y2]
    dfo = pd.merge(shp, old, how='outer', left_on=each, right_index=True)
    dsto = shapefile[:-4] + '_mOage.shp'
    dfo.to_file(dsto)

fig.text(0.28, 0.92, 'Young fraction', fontsize=12)
fig.text(0.71, 0.92, 'Old fraction', ha='center', fontsize=12)
fig.text(0.5, 0.055, 'Median age (yrs) for the complex model', ha='center', fontsize=12)
fig.text(0.095, 0.48, 'Median age (yrs) for the simpler models', va='center', fontsize=12, rotation='vertical')
plt.subplots_adjust(top = .89)
fig.suptitle('Comparison of median ages for young and old water in the glacial aquifer by HUCs', fontsize=16)

# add background color
youngrect = patches.Rectangle((200,70), 760, 830, zorder=-1, alpha=0.5, facecolor='b')
oldrect = patches.Rectangle((960,70), 780, 830, zorder=-1, alpha=0.5, facecolor='r')
fig.patches.append(youngrect)
fig.patches.append(oldrect)

dst = 'YnOages_121_allHUCs'
#loc = os.path.dirname(path[modlist[0]])  # go up one directory to the dir that houses all of the models.
#dst_pth = os.path.join(loc, dst)
dst_pth = os.path.join(fig_dir, dst)
plt.savefig(dst_pth)
#plt.close()
'''

In [ ]:
# 1:1 plot of meanYoungAge and meanOldAge btwn 5LhK against 1L and 5LzK models for ALL HUC scales.

hucs = ['HUC6', 'HUC8', 'HUC10', 'HUC12']
nhucs = len(hucs)
fig1, [[ax1, ax2], [ax3, ax4]] = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
fig2, [[ax5, ax6], [ax7, ax8]] = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
youngaxisdict = {0:ax1, 1:ax2, 2:ax3, 3:ax4}
oldaxisdict = {0:ax5, 1:ax6, 2:ax7, 3:ax8}

for k, each in enumerate(hucs):
    # make sure we get ALL unique HUC IDs
    u1 = dfdict[eptlist[0]][each].unique()
    u2 = dfdict[eptlist[1]][each].unique()
    #u3 = dfdict[eptlist[2]][each].unique()
    u = np.append(u1, u2)
    #u = np.append(u, u3)
    uniques = np.unique(u)
    # remove any HUCs listed in purge_hucs
    for h in purge_hucs:
        ind = np.where(uniques==h)
        uniques = np.delete(uniques, ind)
    youngaxis = youngaxisdict[k]
    oldaxis = oldaxisdict[k]
    ofhucdict = {}
    yfhucdict = {}
    yfvalues = []
    ofvalues = []
    skiphuc = []
    for j, cat_val in enumerate(uniques):  # each HUC ID
        yfmoddict = {} 
        ofmoddict = {} 
        for i, md in enumerate(eptlist):  # each of the 3 FWP models
            mn = eptlist[i]
            df = dfdict[md].loc[dfdict[md][each]==cat_val]
            if df.rt.count() >= minptl:
                youngdf = df.loc[df.rt < age_cutoff]
                olddf = df.loc[df.rt >= age_cutoff]
                yfage = youngdf.rt.median()
                ofage = olddf.rt.median()
                yfmoddict[mn] = yfage
                yfvalues.append(yfage) 
                ofmoddict[mn] = ofage
                ofvalues.append(ofage)
                # pull out raw time info for generalized model areas for later use...
                if (cat_val == float(genHUCdict['Oconto'])) and ('fwp5lzk' in mn):  # 5lzK is the most similar to Generalized models
                    FWPzKocontoDF = df.copy()
                #elif cat_val == float(genHUCdict['TWR'])  and (md == 'fwp5lzk'):
                elif cat_val == float(genHUCdict['TWR'])  and ('fwp5lzk' in mn):
                    FWPzKtwrDF = df.copy()
                elif (cat_val == float(genHUCdict['Oconto'])) and ('fwp5lhk' in mn):  # 5lzK is the most similar to Generalized models
                    FWPhKocontoDF = df.copy()
                elif cat_val == float(genHUCdict['TWR'])  and ('fwp5lhk' in mn):
                    FWPhKtwrDF = df.copy()
                elif (cat_val == float(genHUCdict['Oconto'])) and ('fwp1l' in mn):  # 5lzK is the most similar to Generalized models
                    FWP1locontoDF = df.copy()
                elif cat_val == float(genHUCdict['TWR'])  and ('fwp1l' in mn):
                    FWP1ltwrDF = df.copy()                    
            else:
                skiphuc.append(cat_val)
                break
        if cat_val not in skiphuc:
            yfhucdict[cat_val] = yfmoddict
            ofhucdict[cat_val] = ofmoddict            
            
    young = pd.DataFrame(yfhucdict).T
    old = pd.DataFrame(ofhucdict).T
    young.dropna(axis='index', how='any', inplace=True)  # better way to remove NANs
    old.dropna(axis='index', how='any', inplace=True)
    #for n in modlist:
    #    young = young.loc[~young[n].isnull()]  # need to remove any NANs
    #    old = old.loc[~old[n].isnull()]  # need to remove any NANs
    r2_y1y = r2_score(np.array(young[x1]), np.array(young[y1]))
    #r2_y2y = r2_score(np.array(young[x1]), np.array(young[y2]))
    r2_y1o = r2_score(np.array(old[x1]), np.array(old[y1]))
    #r2_y2o = r2_score(np.array(old[x1]), np.array(old[y2]))
    
    #plotting
    miny, maxy = min(yfvalues), max(yfvalues)
    mino, maxo = min(ofvalues), max(ofvalues)
    #xplot = mini+(maxi-mini)/2
    yrange_y = maxy-miny
    yrange_o = maxo-mino
    
    if k == 0:
        young.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', label='1-layer zoned K', ax=youngaxis)
        #young.plot(kind='scatter', x=x1, y=y2, marker='^', color='blue', label = '5-layer zoned K', ax=youngaxis)        
        old.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', label='1-layer zoned K', ax=oldaxis)
        #old.plot(kind='scatter', x=x1, y=y2, marker='^', color='blue', label = '5-layer zoned K', ax=oldaxis)  
        #ax1.text(miny, miny+0.80*yrange_y, 'R2 = {:3.3f}'.format(r2_y1y), fontsize=9, color='green')
        #ax1.text(miny, miny+0.75*yrange_y, 'R2 = {:3.3f}'.format(r2_y2y), fontsize=9, color='blue')
        #ax5.text(miny, miny+0.80*yrange_y, 'R2 = {:3.3f}'.format(r2_y1y), fontsize=9, color='green')
        #ax5.text(miny, miny+0.75*yrange_y, 'R2 = {:3.3f}'.format(r2_y2y), fontsize=9, color='blue')
        youngaxis.text(miny, miny+0.83*yrange_y, 'R2 = {:3.3f}'.format(r2_y1y), fontsize=9, color='green')
        #youngaxis.text(miny, miny+0.78*yrange_y, 'R2 = {:3.3f}'.format(r2_y2y), fontsize=9, color='blue')
        oldaxis.text(mino, mino+0.83*yrange_o, 'R2 = {:3.3f}'.format(r2_y1o), fontsize=9, color='green')
        #oldaxis.text(mino, mino+0.78*yrange_o, 'R2 = {:3.3f}'.format(r2_y2o), fontsize=9, color='blue')
    else:
        young.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', ax=youngaxis)
        #young.plot(kind='scatter', x=x1, y=y2, marker='^', color='blue', ax=youngaxis)
        old.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', ax=oldaxis)
        #old.plot(kind='scatter', x=x1, y=y2, marker='^', color='blue', ax=oldaxis)     
        youngaxis.text(miny, miny+yrange_y, 'R2 = {:3.3f}'.format(r2_y1y), fontsize=9, color='green')
        #youngaxis.text(miny, miny+0.95*yrange_y, 'R2 = {:3.3f}'.format(r2_y2y), fontsize=9, color='blue')
        oldaxis.text(mino, mino+yrange_o, 'R2 = {:3.3f}'.format(r2_y1o), fontsize=9, color='green')
        #oldaxis.text(mino, mino+0.95*yrange_o, 'R2 = {:3.3f}'.format(r2_y2o), fontsize=9, color='blue')
        
    youngaxis.plot((miny, maxy), (miny, maxy), 'k--')
    oldaxis.plot((mino, maxo), (mino, maxo), 'k--')

    youngaxis.set_xlabel(''), youngaxis.set_ylabel(''), oldaxis.set_xlabel(''), oldaxis.set_ylabel('')
    youngaxis.set_title(each)
    oldaxis.set_title(each)
    
    # Add an attribute to the HUC shapefile
    shapefile = HUCshpdict[each]
    shp = gpd.read_file(shapefile)  
    shp[each] = shp[each].astype('int64')  # convert so can merge
    
    young.index = young.index.astype('int64')  # convert so can merge
    young['mYage_{}-{}'.format(scenario_name, base_case)] = young[x1] - young[y1]
    #young['mYage_hK-zK'] = young[x1] - young[y2]
    df = pd.merge(shp, young, how='outer', left_on=each, right_index=True)
    dst = shapefile[:-4] + '_mYage_{}.shp'.format(scenario_name)
    df.to_file(dst)
    
    old.index = old.index.astype('int64')  # convert so can merge
    old['mOage_{}-{}'.format(scenario_name, base_case)] = old[x1] - old[y1]
    #old['mOage_hK-zK'] = old[x1] - old[y2]
    dfo = pd.merge(shp, old, how='outer', left_on=each, right_index=True)
    dsto = shapefile[:-4] + '_mOage_{}.shp'.format(scenario_name)
    dfo.to_file(dsto)

fig1.text(0.46, 0.92, 'Young fraction', fontsize=12)
#fig.text(0.71, 0.92, 'Old fraction', ha='center', fontsize=12)
fig2.text(0.48, 0.92, 'Old fraction', fontsize=12)
fig1.text(0.5, 0.055, 'Median age (yrs) for the base model', ha='center', fontsize=12)
fig1.text(0.001, 0.48, 'Median age (yrs) for the scenario model', va='center', fontsize=12, rotation='vertical')
fig2.text(0.5, 0.055, 'Median age (yrs) for the base model', ha='center', fontsize=12)
fig2.text(0.005, 0.48, 'Median age (yrs) for the scenario models', va='center', fontsize=12, rotation='vertical')
plt.subplots_adjust(top = .89)
fig1.suptitle('Comparison of median ages for young fraction in the glacial aquifer by HUCs for scenario {}'.format(scenario_name), fontsize=16)
fig2.suptitle('Comparison of median ages for old fraction in the glacial aquifer by HUCs for scenario {}'.format(scenario_name), fontsize=16)

# add background color
#youngrect = patches.Rectangle((200,70), 760, 830, zorder=-1, alpha=0.5, facecolor='b')
#oldrect = patches.Rectangle((960,70), 780, 830, zorder=-1, alpha=0.5, facecolor='r')
#fig.patches.append(youngrect)
#fig.patches.append(oldrect)

dst1 = 'Yages_121_allHUCs_{}'.format(scenario_name)
dst2 = 'Oages_121_allHUCs_{}'.format(scenario_name)
#loc = os.path.dirname(path[modlist[0]])  # go up one directory to the dir that houses all of the models.
#dst_pth = os.path.join(loc, dst)
dst_pth1 = os.path.join(fig_dir, dst1)
dst_pth2 = os.path.join(fig_dir, dst2)
fig1.savefig(dst_pth1)
fig2.savefig(dst_pth2)
#plt.close()